# Bleach corrections evaluation
this notebook evaluate different bleach correction methods
- single exponential fit
- double exponential fit
- different regressions (robust)

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import pynapple as nap

import matplotlib.pyplot as plt

import iblphotometry.preprocessing

from utils import * # don't

from one.api import ONE

one = ONE(base_url="https://alyx.internationalbrainlab.org", mode='remote')

In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 166

%load_ext autoreload
%autoreload 2

In [3]:
# select and load dataset
eid = "974a8a32-2156-4250-b9db-4298fc8daa65"
data_path = Path("/home/georg/data_local/raw_photometry_NE.pqt")
trials = one.load_dataset(eid, "alf/task_00/_ibl_trials.table.pqt")

# load trial data
raw_photometry = pd.read_parquet(data_path)
raw_photometry = nap.TsdFrame(raw_photometry.set_index("times"))

# restricting the fluorescence data to the time within the task +-1s
t_start = trials.iloc[0]["intervals_0"] - 1
t_stop = trials.iloc[-1]["intervals_1"] + 1

session_interval = nap.IntervalSet(t_start, t_stop)
raw_photometry = raw_photometry.restrict(session_interval)



/home/georg/anaconda3/envs/ibl/lib/python3.11/site-packages/one/api.py:300: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self._cache[table].loc[records.index[to_update], :] = records[to_update]
/home/georg/anaconda3/envs/ibl/lib/python3.11/site-packages/one/api.py:300: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  self._cache[table].loc[records.index[to_update], :] = records[to_update]


In [44]:
F = raw_photometry['raw_calcium']
y, t = F.values, F.times()

from bleach_corrections import ExponDecayBleachingModel, DoubleExponDecayBleachingModel

bleaching_model = ExponDecayBleachingModel(method='L-BFGS-B')
bleaching_model._fit(y, t)
y_hat = bleaching_model._predict(t)

fig, axes = plt.subplots()
axes.plot(F)
axes.plot(t, y_hat)

# print(bleaching_model.calc_model_stats(F, use_kde=False, n_samples=-1))



In [25]:
fig, axes = plt.subplots()
axes.plot(t, F.values - y_hat,lw=1)
ax = plt.twinx(axes)
ax.plot(t, (F.values - y_hat)/y_hat, lw=1, color='r', alpha=0.5)

In [46]:
from iblphotometry import utils
F = raw_photometry['raw_calcium']
F = nap.Tsd(t=F.times(), d=F.values / F[-1])
plt.plot(F)


In [48]:


bleaching_model = DoubleExponDecayBleachingModel()
F_corr = bleaching_model.bleach_correct(F, mode='divide')
fig, axes = plt.subplots()
axes.plot(F_corr, lw=0.5, alpha=0.5, color='g')
print(bleaching_model.popt)

bleaching_model = DoubleExponDecayBleachingModel()
F_corr = bleaching_model.bleach_correct(F, mode='subtract')
ax = plt.twinx(axes)
ax.plot(F_corr, lw=0.5, alpha=0.5, color='r')




[4.73352749e-03 1.72255065e+03 4.70016832e-03 8.61275407e+02
 9.99980080e-01 2.23509706e-04]


In [34]:
from iblphotometry import utils
F = raw_photometry['raw_calcium']
F

Time (s)
--------------  ----------
3.635966979     0.00397687
3.66927832      0.0039671
3.702621661     0.00396951
3.735965001     0.00397281
3.769276343     0.00397103
3.802619683     0.00396723
3.835931025     0.003969
...
5160.147930698  0.00393235
5160.181274038  0.00393387
5160.21458538   0.00393336
5160.247928721  0.00393374
5160.281272061  0.00393552
5160.314583403  0.00393412
5160.347926743  0.00393159
dtype: float64, shape: (154710,)

In [ ]:

# bleaching_model = DoubleExponDecayBleachingModel()
# _ = bleaching_model.bleach_correct(F)
# print(bleaching_model.calc_model_stats(F, use_kde=False, n_samples=-1))